In [2]:

import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Outdoors_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [5]:
#filtered to create a DataFrame or table where there are 20 or more total votes
filtered_df = df.filter(df.total_votes >= 20)
filtered_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   30222858|R2FP3U4NHNFNL2|B00YPISPNC|     468413405|Stanley Classic V...|        Outdoors|          5|           25|         29|   N|                Y|GREAT GROWLER! NO...|This item is awes...| 2015-08-31|
|         US|   35677754|R1UUK1977O38MU|B00T8NEI3A|      32341693|Camping Wood Burn...|        Outdoors|          5|    

In [7]:
#filtered to create a DataFrame or table where the percentage of helpful_votes is equal to or greater than 50% 
filtered_df2 = filtered_df.filter((filtered_df.helpful_votes / filtered_df.total_votes) >= 0.5)
filtered_df2.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   30222858|R2FP3U4NHNFNL2|B00YPISPNC|     468413405|Stanley Classic V...|        Outdoors|          5|           25|         29|   N|                Y|GREAT GROWLER! NO...|This item is awes...| 2015-08-31|
|         US|   35677754|R1UUK1977O38MU|B00T8NEI3A|      32341693|Camping Wood Burn...|        Outdoors|          5|    

In [10]:
#filtered to create a DataFrame or table where there is a Vine review
vine_reviews = filtered_df2.filter(filtered_df2["vine"] == "Y")
vine_reviews.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43335941|R3KPC0NBUDASX3|B00R8KC02Q|     872035750|Thule EnRoute Tri...|        Outdoors|          5|           25|         25|   Y|                N|Love the Thule ba...|THULE:<br /><br /...| 2015-08-24|
|         US|   36470546|R119P2A95GGXX4|B00NOYKVSK|     165748383|Wenzel Temp Contr...|        Outdoors|          5|    

In [11]:
#filtered to create a DataFrame or table where there is no Vine review
novine_reviews = filtered_df2.filter(filtered_df2["vine"] == "N")
novine_reviews.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   30222858|R2FP3U4NHNFNL2|B00YPISPNC|     468413405|Stanley Classic V...|        Outdoors|          5|           25|         29|   N|                Y|GREAT GROWLER! NO...|This item is awes...| 2015-08-31|
|         US|   35677754|R1UUK1977O38MU|B00T8NEI3A|      32341693|Camping Wood Burn...|        Outdoors|          5|    

In [12]:
# number of vine reviews
vine_reviews.count()

107

In [16]:
#5-star vine reviews
five_starvine = vine_reviews.filter(vine_reviews.star_rating == 5)
five_starvine.count()

56

In [17]:
# % of five star vine reviews
five_starvine.count() / vine_reviews.count()

0.5233644859813084

In [18]:
# of non-vine reviews
novine_reviews.count()

39869

In [19]:
#5-star non-vine reviews
five_starnovine = novine_reviews.filter(novine_reviews.star_rating == 5)
five_starnovine.count()

21005

In [20]:
# % of five star non-vine reviews
five_starnovine.count() / novine_reviews.count()

0.5268504351751988